In [ ]:
import os
from PIL import Image
# Put your imports here

## Put the name of the augmentation sheet here as a title

In [ ]:
# Create a function that implements the augmentation technique in a code section like this
# The function must be dynamic i.e. it can have multiple differnt values to augment

## Applying the transformations

In [ ]:
def save_image(image, original_name, suffix, output_dir):
    base_name, ext = os.path.splitext(original_name)
    new_name = f"{base_name}_{suffix}{ext}"
    image.save(os.path.join(output_dir, new_name))

input_dir = '../images'
output_dir = 'augmented_images'

image_files = [f for f in os.listdir(input_dir) if f.endswith(('.jpeg'))]

for image_file in image_files:
    image_path = os.path.join(input_dir, image_file)
    image = Image.open(image_path)
    
    # Transformations

    for suffix, img in transformed_images.items():
        save_image(img, image_file, suffix, output_dir)